# Run the CICI SAFE vSDX

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME='pruth-jupyter'
export SSH_PRIVATE_KEY=${HOME}/.ssh/id_rsa

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
# NODE_RESERVATION_ID must be for Skylake nodes
export SDX_RESERVATION_ID='a9f90471-ac8e-447c-bcd0-41e52d11ff49'
export RIAK_RESERVATION_ID='a9f90471-ac8e-447c-bcd0-41e52d11ff49'

#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter/SDX

#### Start RIAK Server

In [ ]:
cd $WORKING_DIR

RIAK_NETWORK='sharednet1'
RIAK_NODE_NAME=${USERNAME}-riak
RIAK_STACK_NAME=${USERNAME}-riak1

echo Creating RIAK server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template riak.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${SDX_RESERVATION_ID}" \
        --parameter "network_name=${RIAK_NETWORK}" \
        --parameter "riak_node_name=${RIAK_NODE_NAME}" \
        --wait \
        ${RIAK_STACK_NAME}

In [ ]:
echo RIAK server creation complete! 
echo ${RIAK_NODE_NAME} status `openstack server show  --format value -c status ${RIAK_NODE_NAME}`

RIAK_PUBLIC_IP=`openstack server show  --format value -c addresses ${RIAK_NODE_NAME} | cut -d " " -f 2`
echo The RIAK se public IP is $RIAK_PUBLIC_IP

In [ ]:
cd $WORKING_DIR

SDX_NETWORK='sharednet1'
SDX_NODE_NAME=${USERNAME}-sdx
SDX_STACK_NAME=${USERNAME}-sdx1

echo Creating SDX server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template sdx.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${SDX_RESERVATION_ID}" \
        --parameter "network_name=${SDX_NETWORK}" \
        --parameter "sdx_node_name=${SDX_NODE_NAME}" \
        --parameter "riak_ip=${RIAK_PUBLIC_IP}" \
        --wait \
        ${SDX_STACK_NAME}

In [ ]:
echo SDX server creation complete! 
echo ${SDX_NODE_NAME} status `openstack server show  --format value -c status ${SDX_NODE_NAME}`

SDX_PUBLIC_IP=`openstack server show  --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo The SDX se public IP is $SDX_PUBLIC_IP

In [ ]:
SDX_PUBLIC_IP=`openstack server show --format value -c addresses ${SDX_NODE_NAME} | cut -d " " -f 2`
echo SDX_NODE_NAME is $SDX_PUBLIC_IP

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${SDX_PUBLIC_IP} \
    sudo tail /root/boot.log


# Please Cleanup Your Resources!!!

#### Delete SDX Stack

In [ ]:
echo Deleting stack ${SDX_STACK_NAME}
openstack stack delete -y --wait ${SDX_STACK_NAME}
echo Stack ${SDX_STACK_NAME} deleted

#### Delete RIAK Stack

In [ ]:
echo Deleting stack ${RIAK_STACK_NAME}
openstack stack delete -y --wait ${RIAK_STACK_NAME}
echo Stack ${RIAK_STACK_NAME} deleted